# Route analysis using Batfish applied to Elecnet 

In [19]:
# Importing required libraries, setting up logging, and loading questions
import logging
import random  # noqa: F401
from typing import List, Optional  # noqa: F401

import pandas as pd
from IPython.display import display
from pandas.io.formats.style import Styler

from pybatfish.client.session import Session  # noqa: F401

# noinspection PyUnresolvedReferences
from pybatfish.datamodel import Edge, Interface  # noqa: F401
from pybatfish.datamodel.answer import TableAnswer
from pybatfish.datamodel.flow import HeaderConstraints, PathConstraints  # noqa: F401
from pybatfish.datamodel.route import BgpRoute  # noqa: F401
from pybatfish.util import get_html

# Configure all pybatfish loggers to use WARN level
logging.getLogger("pybatfish").setLevel(logging.WARN)

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option("display.html.use_mathjax", False)

# UUID for CSS styles used by pandas styler.
# Keeps our notebook HTML deterministic when displaying dataframes
_STYLE_UUID = "pybfstyle"


class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)


def show(df):
    """
    Displays a dataframe as HTML table.

    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """
    if isinstance(df, TableAnswer):
        df = df.frame()

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    display(
        MyStyler(df)
        .set_uuid(_STYLE_UUID)
        .format(get_html)
        .set_properties(**{"text-align": "left", "vertical-align": "top"})
    )

bf = Session(host="localhost")

In [20]:
# Assign a friendly name to your network and snapshot
NETWORK_NAME = "example_notwork"
SNAPSHOT_NAME = "example_snipshot"

# this is the path to the network. You need to spell it correctly
SNAPSHOT_PATH = "networks/elecnet"

# Now create the network and initialize the snapshot
bf.set_network(NETWORK_NAME)
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snipshot'

In [21]:
# Get routing tables for all nodes and VRFs
routes_all = bf.q.routes().answer().frame()
routes_all

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,adlcore-1,default,10.10.2.96/30,interface eth0 ip 10.21.9.194,10.21.9.194,eth0,ospf,20,110,None
1,adlcore-1,default,10.10.9.204/30,interface eth0 ip 10.21.9.194,10.21.9.194,eth0,ospf,20,110,None
2,adlcore-1,default,10.10.10.196/30,interface eth4,AUTO/NONE(-1l),eth4,connected,0,0,None
3,adlcore-1,default,10.10.22.24/30,interface eth3,AUTO/NONE(-1l),eth3,connected,0,0,None
4,adlcore-1,default,10.10.45.128/30,interface eth3 ip 10.10.22.26,10.10.22.26,eth3,ospf,20,110,None
...,...,...,...,...,...,...,...,...,...,...
12541,yrv-gw,default,10.25.243.0/25,interface eth0 ip 10.22.217.197,10.22.217.197,eth0,ospf,30,110,None
12542,yrv-gw,default,10.25.243.0/25,interface eth1 ip 10.22.221.180,10.22.221.180,eth1,ospf,30,110,None
12543,yrv-gw,default,10.25.250.0/25,interface eth0 ip 10.22.217.197,10.22.217.197,eth0,ospf,30,110,None
12544,yrv-gw,default,10.25.250.0/25,interface eth1 ip 10.22.221.180,10.22.221.180,eth1,ospf,30,110,None


In [16]:
# Get the routing table for the 'default' VRF on border routers of as1
# using Pandas filtering

# In IP-based computer networks, virtual routing and forwarding (VRF) is a 
# technology that allows multiple instances of a routing table to co-exist
# within the same router at the same time.
routes_wwanode = routes_all[(routes_all['Node'].str.contains('ww'))]
routes_wwanode

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
270,wwanode0,default,0.0.0.0/0,interface eth0 ip 10.25.165.206,10.25.165.206,eth0,static,0,1,None
271,wwanode0,default,10.25.165.128/25,interface eth0,AUTO/NONE(-1l),eth0,connected,0,0,None
272,wwanode0,default,10.25.165.165/32,interface eth0,AUTO/NONE(-1l),eth0,local,0,0,None
273,wwanode1,default,0.0.0.0/0,interface eth0 ip 10.25.160.146,10.25.160.146,eth0,static,0,1,None
274,wwanode1,default,10.25.160.0/24,interface eth0,AUTO/NONE(-1l),eth0,connected,0,0,None
275,wwanode1,default,10.25.160.57/32,interface eth0,AUTO/NONE(-1l),eth0,local,0,0,None
276,wwanode2,default,0.0.0.0/0,interface eth0 ip 10.25.163.79,10.25.163.79,eth0,static,0,1,None
277,wwanode2,default,10.25.163.0/24,interface eth0,AUTO/NONE(-1l),eth0,connected,0,0,None
278,wwanode2,default,10.25.163.44/32,interface eth0,AUTO/NONE(-1l),eth0,local,0,0,None


In [13]:
# Getting BGP routes in the routing table for the 'default' VRF on border routers of as1
# using BF parameters
routes_wwanode_static = bf.q.routes(nodes="/wwanode/", vrfs="default", protocols="static").answer().frame()
routes_wwanode_static

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,wwanode0,default,0.0.0.0/0,interface eth0 ip 10.25.165.206,10.25.165.206,eth0,static,0,1,None
1,wwanode1,default,0.0.0.0/0,interface eth0 ip 10.25.160.146,10.25.160.146,eth0,static,0,1,None
2,wwanode2,default,0.0.0.0/0,interface eth0 ip 10.25.163.79,10.25.163.79,eth0,static,0,1,None


In [18]:
routes_filtered = routes_all[(routes_all['Protocol'] == 'static') & (routes_all['Metric'] < 50)]
routes_filtered

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,brwnode0,default,0.0.0.0/0,interface eth0 ip 10.25.11.96,10.25.11.96,eth0,static,0,1,None
3,brwnode1,default,0.0.0.0/0,interface eth0 ip 10.25.15.38,10.25.15.38,eth0,static,0,1,None
6,cbpdnode0,default,0.0.0.0/0,interface eth0 ip 10.25.68.57,10.25.68.57,eth0,static,0,1,None
9,cbpdnode1,default,0.0.0.0/0,interface eth0 ip 10.25.69.164,10.25.69.164,eth0,static,0,1,None
12,cmtnode0,default,0.0.0.0/0,interface eth0 ip 10.25.152.118,10.25.152.118,eth0,static,0,1,None
...,...,...,...,...,...,...,...,...,...,...
264,wlwnode0,default,0.0.0.0/0,interface eth0 ip 10.25.21.72,10.25.21.72,eth0,static,0,1,None
267,wlwnode1,default,0.0.0.0/0,interface eth0 ip 10.25.22.13,10.25.22.13,eth0,static,0,1,None
270,wwanode0,default,0.0.0.0/0,interface eth0 ip 10.25.165.206,10.25.165.206,eth0,static,0,1,None
273,wwanode1,default,0.0.0.0/0,interface eth0 ip 10.25.160.146,10.25.160.146,eth0,static,0,1,None
